In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
orders = pd.read_csv('data/Orders.csv')

In [3]:
orders['time'] = pd.date_range('2012-10-01', periods=len(orders), freq='1min')

In [4]:
# orders.to_csv('Orders_with_time', index=False)

In [87]:
products = pd.read_csv('data/Products.csv')
products.head()

,Name,BarCode,Price,LoaltyCoeff,Point_id
0,Chocolate Sandwich Cookies,903304753514,21.60,0.99,6191
1,All-Seasons Salt,401309075522,25.90,0.58,6870
2,Robust Golden Unsweetened Oolong Tea,851892036179,8.47,0.66,406
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,445454311328,13.53,0.77,5459
4,Green Chile Anytime Sauce,304753955703,22.73,0.24,4954


In [88]:
orders_product = orders.merge(products, left_on='product_id', left_index=False, right_index=True)

In [89]:
orders_product.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_last_order,product_id,reordered,time,Name,BarCode,Price,LoaltyCoeff,Point_id
0,2539329,1,1,2,8,NaN,196,0,2012-10-01 00:00:00,Cold Brew Coffee Tahitian Vanilla,284125606973,16.19,0.39,7309
5,2398795,1,2,3,7,15.0,196,1,2012-10-01 00:05:00,Cold Brew Coffee Tahitian Vanilla,284125606973,16.19,0.39,7309
11,473747,1,3,3,12,21.0,196,1,2012-10-01 00:11:00,Cold Brew Coffee Tahitian Vanilla,284125606973,16.19,0.39,7309
16,2254736,1,4,4,7,29.0,196,1,2012-10-01 00:16:00,Cold Brew Coffee Tahitian Vanilla,284125606973,16.19,0.39,7309
21,431534,1,5,4,15,28.0,196,1,2012-10-01 00:21:00,Cold Brew Coffee Tahitian Vanilla,284125606973,16.19,0.39,7309


In [90]:
def group_data_by_months(data):
    X, y = [], []
    last_order_time = data.time[0]
    sum_price = 0.
    counter = 0

    for j, row in data.iterrows():
        if j == 1:
            break
        sum_price += row.loc['Price']
        counter += 1
        if (row.time - last_order_time).days >= 5:
            last_order_time = row.time
            X.append([counter, sum_price])
            y.append(sum_price)

            sum_price = 0
            counter = 0
            
    X.pop(-1)
    y.pop(0)
    X.pop(-1)
    y.pop(-1)
        
    return X, y

In [91]:
X, y = group_data_by_months(orders_product[:100000])

In [92]:
def _train_test_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [93]:
X_train, X_test, y_train, y_test = _train_test_split(X, y)

In [94]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train);

In [108]:
model.score(X_test, y_test)

-0.006745277474688338